In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv('data/8_sentences_train.csv',sep='|',index_col=0)
#df_test = pd.read_csv('data/5_sentences_test.csv',sep='|',index_col=0)

/Users/rubenstern/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df_train_pos = df_train[df_train['label']==1]
df_train_neg = df_train[df_train['label']==0].sample(n = df_train_pos.shape[0])

df_array = [df_train_pos,df_train_neg]
df_train_balanced = pd.concat(df_array)
df_train_balanced = df_train_balanced.sample(frac=1)
df_train_balanced = df_train_balanced.fillna("")

In [5]:
all_sentences = df_train_balanced['sentence'] + ' ' + \
                df_train_balanced['prev1'] + ' ' + \
                df_train_balanced['prev2'] + ' ' + \
                df_train_balanced['prev3'] + ' ' + \
                df_train_balanced['prev4'] 
    

In [6]:
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,2))
vectorizer.fit(all_sentences.values.astype('U'))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [7]:
X_sentence = vectorizer.transform(df_train_balanced['sentence'].values.astype('U'))
X_prev1 = vectorizer.transform(df_train_balanced['prev1'].values.astype('U'))
X_prev2 = vectorizer.transform(df_train_balanced['prev2'].values.astype('U'))
X_prev3 = vectorizer.transform(df_train_balanced['prev3'].values.astype('U'))
X_prev4 = vectorizer.transform(df_train_balanced['prev4'].values.astype('U'))

list_sentence = [X_sentence,X_prev1,X_prev2,X_prev3,X_prev4]

In [9]:
df_train_balanced.head()

,prev8,prev7,prev6,prev5,prev4,prev3,prev2,prev1,sentence,label
3271045,"That's bad, stupid people need to die.",Any idea if these kids had a license to ill?,"In other news, liberals are conservatives who ...",Don't forget that Trump believes that vaccines...,I think the best part is that the science isn'...,The real question here is if the possibility o...,"It was at this moment, Kristen knew she fucked...",WTF is wrong with them being a case of an argu...,"Goddamn, she even looks like a an anti-vaxxer.",0
247268,,,,,,,,My favorite memory of your city :-),Genau mein Bier!,1
1411892,,,,,,,,My wife was 100% convinced that there is no po...,Is your wife a cute black child that forgets t...,0
1381420,Makaa?,That is one crazy bird.,"Y'know, the thing about a Potoo, he's got life...",That thing stares into my soul.,so cute,It's kinda cute... ^Kinda,They are scary as shit.,GET THE SALT SAMMY!,It scared the shit out of me.,0
5643302,,,TRIGGERED,someone pull out the attack helicopter copypasta,nice meme,Lel good thing enleat isn't a mod anymore and ...,What's the joke?,pockets are inferior anyway,ACTUALLY I SEXUALLY IDENTIFY AS A GHASTLY GIBU...,1


In [10]:
X = hstack(list_sentence)

y = df_train_balanced['label']
X_train, X_test, y_train, y_test,raw_train,raw_test = train_test_split(X, y,df_train_balanced, test_size=0.2)


In [25]:
raw_test = raw_test[['prev4','prev3','prev2','prev1','sentence','label']]

In [12]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_test_predict = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_test_predict)
print('accuracy: ',accuracy)

accuracy:  0.742355812657


In [34]:
#false negative: predicted negative
for index,row in raw_test[(y_test_predict==0) & (y_test==1)].head(30).iterrows():
    print('prev4',row['prev4'])
    print('prev3',row['prev3'])
    print('prev2',row['prev2'])
    print('prev1',row['prev1'])
    print('sentence',row['sentence'])
    print()

prev4 You're only saying that because you're white and you've never had to walk a mile in a the shoes of a minority
prev3 It's called affirmative action
prev2 "*Claims* equality for all" =/= Equality for all
prev1 I'm not sure of the source, but according to my sociology professor less than 1% of all scholarships are set aside for scholarships specific to race
sentence Man being white in America is so fucking hard, why I cant I be an inner city kid and get free money.

prev4 What the shit is happening in this comment section?
prev3 Thanks, I'm sure that was her goal.
prev2 Holy shit.
prev1 So the analogy holds up.
sentence lol, yea kill innocent ppl cause you are mad

prev4 Holy crap..I'm gonna go check my car and house now.
prev3 I suppose he was... on a list.
prev2 THEY CAN SMELL YOUR CUM
prev1 Brb checking for FBI tracking devices
sentence 'Murica, the land of freedom

prev4 The Middle East, I could live out the rest of my life there and die not having a clue as to what the fuck is 

In [36]:
#false positive: predicted positive
for index,row in raw_test[(y_test_predict==1) & (y_test==0)].head(30).iterrows():
    print('prev4',row['prev4'])
    print('prev3',row['prev3'])
    print('prev2',row['prev2'])
    print('prev1',row['prev1'])
    print('sentence',row['sentence'])
    print()

prev4 
prev3 
prev2 
prev1 My wife was 100% convinced that there is no possible way to open this package without spilling the tomatoes everywhere.
sentence Is your wife a cute black child that forgets to put water in his noodles?

prev4 I know it's wrong but all I can see is Stewie.
prev3 Plot twist: She was the Sheriff.
prev2 As descriptive as a mute retard attempting to describe a biscuit.
prev1 guns don't kill people, 2 year olds kill people
sentence And that, America, is the reason why you need to get rid of the NRA and all civilian-owned guns.

prev4 My Microsoft Surface Pro 4.
prev3 My hair colors :D
prev2 The scar on the side of my head.
prev1 I made a BB-8 dress
sentence Me being single.

prev4 Haha...I get it
prev3 Git Gid
prev2 0/10 missing the ACOG.
prev1 Problem is that Siege has not implemented bullet spread - from a still standing position, you will always fire true, whereas in real life guns with a short barrel like this will have a relatively wide cone of fire at range.